An [attractor](https://en.wikipedia.org/wiki/Attractor#Strange_attractor) is a set of numerical values to which a numerical system tends to evolve. An attractor is called a [strange attractor](https://en.wikipedia.org/wiki/Attractor#Strange_attractor) if the resulting pattern has a fractal structure.

# Clifford Attractors

For example, a [Clifford Attractor](http://paulbourke.net/fractals/clifford) is a strange attractor defined by two iterative equations that determine discrete steps in the path of a particle across a 2D space, given a starting point _(x0,y0)_ and the values of four parameters _(a,b,c,d)_:

\begin{equation}
x_{n +1} = \sin(a y_{n}) + c \cos(a x_{n})\\
y_{n +1} = \sin(b x_{n}) + d \cos(b y_{n})
\end{equation}

At each time step, the equations define the location for the following time step, and the accumulated locations show the areas of the 2D plane most commonly visited by the imaginary particle.  

It's easy to calculate these values in Python using [Numba](http://numba.pydata.org) and to visualize them with [Datashader](http://datashader.org), using code and parameters adapted from [L&aacute;zaro Alonso](https://lazarusa.github.io/Webpage/codepython2.html), [François Pacull](https://aetperf.github.io/2018/08/29/Plotting-Hopalong-attractor-with-Datashader-and-Numba.html), and [Jason Rampe](https://softologyblog.wordpress.com/2017/03/04/2d-strange-attractors).

First, we define the iterative attractor equation:

In [ ]:
import numpy as np, pandas as pd, datashader as ds
from datashader import transfer_functions as tf
from datashader.colors import inferno, viridis
from numba import jit

@jit
def clifford(a, b, c, d, x, y):
    return np.sin(a * y) + c * np.cos(a * x), \
           np.sin(b * x) + d * np.cos(b * y)

We then evaluate this equation many times, creating a set of (x,y) coordinates visited. The @jit here and above is optional, but it makes the code 50x faster.

In [ ]:
n=10000000

@jit
def trajectory(fn, a, b, c, d, x0=0, y0=0, n=n):
    x, y = np.zeros(n), np.zeros(n)
    x[0], y[0] = x0, y0
    for i in np.arange(n-1):
        x[i+1], y[i+1] = fn(a, b, c, d, x[i], y[i])
    return pd.DataFrame(dict(x=x,y=y))

In [ ]:
%%time
df = trajectory(clifford, -1.3, -1.3, -1.8, -1.9)

In [ ]:
df.tail()

We can now aggregate these coordinates into a 2D grid:

In [ ]:
%%time

cvs = ds.Canvas(plot_width = 700, plot_height = 700)
agg = cvs.points(df, 'x', 'y')

So that we can see the grid, we need to turn the integer count-per-cell values into colors, such as greyscale values from white to black:

In [ ]:
ds.transfer_functions.Image.border=0

tf.shade(agg, cmap = ["white", "black"])

As you can see, the most-visited areas of the plane have an interesting structure for this set of parameters. Let's collect some other suitable [white-background, uniform sequential colormaps](http://holoviews.org/user_guide/Colormaps.html) for subsequent plots:

In [ ]:
from colorcet import palette
cmaps =  [palette[p][::-1] for p in ['bgy', 'bmw', 'bgyw', 'bmy','fire', 'gray', 'kbc', 'kgy']]
cmaps += [inferno[::-1], viridis[::-1]]

You can get a variety of trajectories if you use different parameter values, and a variety of different appearances depending on colormap:

In [ ]:
def plot(fn, *args, **kw):
    num   = kw.pop('n', n)
    cmap  = kw.pop('cmap', viridis)
    label = kw.pop('label', False)
    traj  = kw.pop('trajectory', trajectory)
    
    label = ("{}, "*(len(args)-1)+" {}").format(*args) if label else None
    df  = traj(fn, *args, n=num)
    cvs = ds.Canvas(plot_width = 400, plot_height = 400)
    agg = cvs.points(df, 'x', 'y')
    img = tf.shade(agg, cmap=cmap, name=label)
    return img

cvals = [
    (-1.3,   -1.3,   -1.8,   -1.9),
    (-1.4,    1.6,    1.0,    0.7),
    ( 1.7,    1.7,    0.6,    1.2),
    ( 1.7,    0.7,    1.4,    2.0),
    (-1.7,    1.8,   -1.9,   -0.4),
    ( 1.1,   -1.32,  -1.03,   1.54),
    (-1.9,   -1.9,   -1.9,   -1.0),
    ( 0.77,   1.99,  -1.31,  -1.45),
    ( 0.75,   1.34,  -1.93,   1.00),
    (-1.32,  -1.65,   0.74,   1.81),
    #( 1.10,  -0.90,   0.10,   0.20),
]

tf.Images(*[plot(clifford, *cvals[i], cmap=cmaps[i%len(cmaps)]) for i in range(len(cvals))]).cols(5)

The above examples are selected for illustration, but randomly sampling the parameter space will show that there are less interesting parameter combinations as well, such as all values being on a single fixed point:

In [ ]:
import numpy.random
numpy.random.seed(12)
rvals=numpy.random.random((5,4))*4-2

tf.Images(*[plot(clifford, *rvals[i], cmap=cmaps[(i+1)%len(cmaps)], label=True) 
            for i in range(len(rvals))]).cols(5)

If you wish, datashader could easily be used to filter out such uninteresting examples, by applying a criterion to the aggregate array before shading and showing only those that remain (e.g. rejecting those where 80% of the pixel bins are empty).


## De Jong attractors

A related set of attractors was proposed by [Peter de Jong](http://paulbourke.net/fractals/peterdejong):

In [ ]:
@jit
def dejong(a, b, c, d, x=0, y=0):
    return np.sin(a * y) - np.cos(b * x), \
           np.sin(c * x) - np.cos(d * y)

dvals = [
    (-1.244, -1.251, -1.815, -1.908),
    ( 1.7,    1.7,    0.6,    1.2),
    ( 1.4,   -2.3,    2.4,   -2.1),
    (-2.7,   -0.09,  -0.86,  -2.2),
    (-0.827, -1.637,  1.659, -0.943),
    (-2.24,   0.43,  -0.65,  -2.43),
    (-2.0,   -2.0,   -1.2,    2.0),
    (-0.709,  1.638,  0.452,  1.740),
    ( 2.01,  -2.53,   1.61,  -0.33),
    ( 1.40,   1.56,   1.40,  -6.56)]

tf.Images(*[plot(dejong, *dvals[i], cmap=cmaps[1-i]) for i in range(len(dvals))]).cols(5)

# Svensson attractors

Another variation was provided by Johnny Svensson:

In [ ]:
@jit
def svensson(a, b, c, d, x=0, y=0):
    return d * np.sin(a * x) - np.sin(b * y), \
           c * np.cos(a * x) + np.cos(b * y)

svals = [
    ( 1.5,   -1.8,    1.6,    0.9),
    (-0.91,  -1.251, -1.815, -1.908),
    (-1.78,   1.29,  -0.09,  -1.18),
    (-0.91,  -1.29,  -1.97,  -1.56),
    ( 1.40,   1.56,   1.40,  -6.56)]

tf.Images(*[plot(svensson, *svals[i], cmap=cmaps[i]) for i in range(len(svals))]).cols(5)

# Bedhead Attractor

Another variation, from [Ivan Emrich](https://www.deviantart.com/jaguarfacedman) and [Jason Rampe](https://softologyblog.wordpress.com/2017/03/04/2d-strange-attractors):

In [ ]:
@jit
def bedhead(a, b, c, d, x=0, y=0):
    return np.sin(x*y/b)*y + np.cos(a*x-y), \
           x + np.sin(y)/b

bvals = [#a      b         c  d  x0 y0
    ( 0.65343,  0.7345345, 0, 0, 1, 1),
    (-0.81,    -0.92,      0, 0, 1, 1),
    (-0.64,     0.76,      0, 0, 1, 1),
   #( 0.06,     0.98,      0, 0, 1, 1),
    (-0.67,     0.83,      0, 0, 1, 1)]

tf.Images(*[plot(bedhead, *bvals[i], cmap=cmaps[1-i]) for i in range(len(bvals))]).cols(5)

# Fractal Dream Attractor

Another variation from Clifford A. Pickover, discussed in his book “Chaos In Wonderland”, with parameters from [Jason Rampe](https://softologyblog.wordpress.com/2017/03/04/2d-strange-attractors):

In [ ]:
@jit
def fractaldream(a, b, c, d, x=0, y=0):
    return np.sin(y*b)+c*np.sin(x*b), \
           np.sin(x*a)+d*np.sin(y*a)

fvals = [
    (-0.966918,  2.879879,  0.765145, 0.744728, 0.1, 0.1),
    (-2.9585,   -2.2965,   -2.8829,  -0.1622,   0.1, 0.1),
    (-2.8276,    1.2813,    1.9655,   0.597,    0.1, 0.1),
    (-1.1554,   -2.3419,   -1.9799,   2.1828,   0.1, 0.1),
    (-1.9956,   -1.4528,   -2.6206,   0.8517,   0.1, 0.1)]

tf.Images(*[plot(fractaldream, *fvals[i], cmap=cmaps[i]) for i in range(len(fvals))]).cols(5)

## Hopalong attractors

A different type of attractor was introduced by Barry Martin, here with code for two variants from [François Pacull](https://aetperf.github.io/2018/08/29/Plotting-Hopalong-attractor-with-Datashader-and-Numba.html).  This one has only three parameters; the `d` parameter here is only here for compatibility with the trajectory function:

In [ ]:
@jit
def hopalong_1(a, b, c, d, x, y):
    return y - np.sqrt(np.fabs(b * x - c)) * np.sign(x), \
           a - x

h1vals = [
    (  2.0,    1.0,    0.0,    0),
    (-11.0,    0.05,   0.5,    0),
    (  2.0,    0.05,   2.0,    0),
    (  0.1,    0.1,   20.0,    0),
    (  1.1,    0.5,    1.0,    0)]

tf.Images(*[plot(hopalong_1, *h1vals[i], cmap=cmaps[1-i]) for i in range(len(h1vals))]).cols(5)

In [ ]:
@jit
def hopalong_2(a, b, c, d, x, y):
    return y - 1.0 - np.sqrt(np.fabs(b * x - 1.0 - c)) * np.sign(x - 1.0), \
           a - x - 1.0

h2vals = [
    ( 7.16878197155893, 8.43659746693447,  2.55983412731439, 0),
    ( 7.7867514709942,  0.132189802825451, 8.14610984409228, 0),
    ( 9.74546888144687, 1.56320227775723,  7.86818214459345, 0)]

tf.Images(*[plot(hopalong_2, *h2vals[i], cmap=cmaps[i]) for i in range(len(h2vals))]).cols(5)

#  Symmetric Icon Attractor

The Hopalong equations often result in symmetric patterns, but a different approach is to *force* the patterns to be symmetric, which is often pleasing. Examples from “Symmetry in Chaos” by Michael Field and Martin Golubitsky, with code and parameters from [Jason Rampe](https://softologyblog.wordpress.com/2017/03/04/2d-strange-attractors):

In [ ]:
@jit
def symmetricicon(a, b, g, o, l, d=3, x=0.01, y=0.01):
    zzbar = x*x + y*y
    p = a*zzbar + l
    zreal, zimag = x, y
    
    for i in range(1, d-1):
        za, zb = zreal * x - zimag * y, zimag * x + zreal * y
        zreal, zimag = za, zb
    
    zn = x*zreal - y*zimag
    p += b*zn
    
    return p*x + g*zreal - o*y, \
           p*y - g*zimag + o*x


# Same as standard trajectory but with more arguments 
@jit
def trajectory_si(fn, a, b, g, o, l, d, x0=0.01, y0=0.01, n=n):
    x, y = np.zeros(n), np.zeros(n)
    x[0], y[0] = x0, y0
    for i in np.arange(n-1):
        x[i+1], y[i+1] = fn(a, b, g, o, l, d, x[i], y[i])
    return pd.DataFrame(dict(x=x,y=y))

ivals = [ #a   b      g      o       l      d
    ( 1.8,    0.0,   1.0,   0.1,   -1.93,   5),
    ( 5.0,   -1.0,   1.0,   0.188, -2.5,    5),
    (-1.0,    0.1,  -0.82,  0.12,   1.56,   3),
    ( 1.806,  0.0,   1.0,   0.0,   -1.806,  5),
    (10.0,  -12.0,   1.0,   0.0,   -2.195,  3),
    (-2.5,    0.0,   0.9,   0.0,    2.5,    3),
    ( 3.0,  -16.79,  1.0,   0.0,   -2.05,   9),
    ( 5.0,    1.5,   1.0,   0.0,   -2.7,    6),
    (-2.5,    0.0,   0.9,   0.0,    2.409, 23),
    ( 1.0,   -0.1,   0.167, 0.0,   -2.08,   7),
    ( 2.32,   0.0,   0.75,  0.0,   -2.32,   5),
    (-2.0,    0.0,  -0.5,   0.0,    2.6,    5),
    ( 2.0,    0.2,   0.1,   0.0,   -2.34,   5),
    ( 2.0,    0.0,   1.0,   0.1,   -1.86,   4),
    (-1.0,    0.1,  -0.82,  0.0,    1.56,   3),
    (-1.0,    0.1,  -0.805, 0.0,    1.5,    3),
    (-1.0,    0.03, -0.8,   0.0,    1.455,  3),
    (-2.5,   -0.1,   0.9,  -0.15,   2.39,  16)
]

tf.Images(*[plot(symmetricicon, *ivals[i], cmap=cmaps[(i+1)%len(cmaps)], trajectory=trajectory_si) 
            for i in range(len(ivals))]).cols(5)

## Interactive plotting

If you are running a live Python process, you can use Datashader with HoloViews and Bokeh to zoom in and see the individual steps in any of these calculations:

In [ ]:
import holoviews as hv
from holoviews.operation.datashader import datashade, dynspread
hv.extension('bokeh')

dynspread(datashade(hv.Points(trajectory(clifford, *cvals[5])), cmap=viridis[::-1]).options(width=400,height=400))

Each time you zoom in in a live process, the data will be reaggregated, which should take a small fraction of a second for 10 million points.  Eventually, once you zoom in enough you should see individual data points, as we are not connecting the points into a trajectory here. 

You can also try "connecting the dots", which will reveal how the particle jumps discretely from one region of the space to another:

In [ ]:
dynspread(datashade(hv.Path([trajectory(clifford, *cvals[5])]), cmap=viridis[::-1]).options(width=400,height=400))

Again, if you zoom in on a live server, the plot will update so that you can see the individual traces involved. 

On the live server, you can also explore to find your own parameter values that generate interesting patterns:

In [ ]:
def hv_clif(a,b,c,d,x0=0,y0=0,n=n):
    return datashade(hv.Points(trajectory(clifford, a, b, c, d, x0, y0, n)), cmap=inferno[::-1], dynamic=False)
a,b,c,d=cvals[6]

dm = hv.DynamicMap(hv_clif, kdims=['a', 'b', 'c', 'd'])
dm = dm.redim.range(a=(-2.0, 2.0), b=(-2.0,2.0), c=(-2.0,2.0), d=(-2.0,2.0))
dm = dm.redim.default(a=a, b=b, c=c, d=d).options(width=500,height=500)
dm

Although many of the regions of this four-dimensional parameter space generate uninteresting trajectories such as single points, you can find interesting regions by starting with one of the a,b,c,d tuples of values in previous plots, then click on one slider and use the left and right arrow keys to see how the plot changes as that parameter changes.